In [ ]:
!pip install flask==0.12.2
# running flask app in colab using ngrok
!pip install pyngrok

In [ ]:
# speech recognition module
!pip install SpeechRecognition

In [ ]:
from flask import Flask, render_template, request, redirect
import speech_recognition as sr
from pyngrok import ngrok
import os 

In [ ]:
os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 5000

#Setting an auth token
ngrok.set_auth_token("YOUR_TOKEN")
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

In [ ]:
# Define Flask routes
@app.route("/", methods=["GET", "POST"])
def index():
    transcript = ""
    if request.method == "POST":
        print("FORM DATA RECEIVED")

        if "file" not in request.files:
            return redirect(request.url)

        file = request.files["file"]
        if file.filename == "":
            return redirect(request.url)

        if file:
            recognizer = sr.Recognizer()
            audioFile = sr.AudioFile(file)
            with audioFile as source:
                data = recognizer.record(source)
            transcript = recognizer.recognize_google(data, key=None)

    return render_template('index.html', transcript=transcript)

In [ ]:
# Start the Flask server in a new thread
app.run()